# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [7]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1, 2, 3, 4, 5, 4573]


In [18]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1,
    "name": "Devweekerson",
    "account": {
      "id": 1,
      "number": "01.097954-4",
      "agency": "2030",
      "balance": 624.12,
      "limit": 1000.0
    },
    "card": {
      "id": 1,
      "number": "xxxx xxxx xxxx 1111",
      "limit": 2000.0
    },
    "features": [
      {
        "id": 2,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
        "description": "Pagar"
      },
      {
        "id": 3,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
        "description": "Transferir"
      },
      {
        "id": 4,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
        "description": "Conta Corrente"
      },
      {
        "id": 5,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/cards.svg",
        "description": "Cart\u00f5es"
      },
     

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [9]:
print(users)

[{'id': 1, 'name': 'Devweekerson', 'account': {'id': 1, 'number': '01.097954-4', 'agency': '2030', 'balance': 624.12, 'limit': 1000.0}, 'card': {'id': 1, 'number': 'xxxx xxxx xxxx 1111', 'limit': 2000.0}, 'features': [{'id': 2, 'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg', 'description': 'Pagar'}, {'id': 3, 'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg', 'description': 'Transferir'}, {'id': 4, 'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg', 'description': 'Conta Corrente'}, {'id': 5, 'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/cards.svg', 'description': 'Cartões'}, {'id': 1, 'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pix.svg', 'description': 'PIX'}], 'news': [{'id': 1, 'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg', 'descri

In [16]:
for user in users:
  user['news'].append({

      "description": f"Ola {user['name']} fiquei feliz em saber que voce esta se preparando para um futuro abundante. Hoje voce tem um limite de {user['account']['limit']}. Para mais dicas de investimento entre em contato comigo pelo telefone (99) 99999 - 9999."
  })

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [17]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Devweekerson updated? False!
User Isa updated? True!
